# Lab 4: Advanced Neural Nets
### Шевченко Юлія, ФІ-31мн

### Lab Task:

1. _**Завдання щодо генерації текстів або машинного перекладу (на вибір) на базі рекурентних мереж або трансформерів (на вибір)**_. Вирішити завдання щодо генерації текстів або машинного перекладу. Особливо вітаються україномовні моделі.  
2. Провести експерименти з моделями бібліотеки HF Transformers за допомогою (наприклад) Pipeline модуля.
3. _**Завдання щодо генерації або стилізації зображень (на вибір)**_. Вирішити завдання перенесення стилю або генерації зображень (архітектура за вашим вибором: GAN/DCGAN/VAE).

In [3]:
# All the imports for the task

import os
from urllib.parse import urljoin

import pandas as pd
import requests
from keras import layers
%matplotlib inline
import tensorflow as tf
from tensorflow.keras import layers, models
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np

### Task 3

In [2]:
df = pd.read_csv('../Lab_4/list.csv')
output_folder = '../Lab_4/doodles_images'
os.makedirs(output_folder, exist_ok=True)

for index, row in df.iterrows():
    image_url = row['high_res_url']

    if pd.notna(image_url) and isinstance(image_url, str):
        if not image_url.startswith(('http://', 'https://')):
            image_url = urljoin('https:', image_url)

        image_filename = os.path.join(output_folder, f"{row['year']}_{row['month']}_{row['day']}_{row['name']}.jpg")

        try:
            response = requests.get(image_url, stream=True)
            with open(image_filename, 'wb') as file:
                for chunk in response.iter_content(chunk_size=1024):
                    if chunk:
                        file.write(chunk)
        except Exception as e:
            print('Error, while downloading {}: {}'.format(image_url, e))
    else:
        print('Error, invalid URL:'.format(image_url))

print('Images were downloaded to {}'.format(output_folder))

Images were downloaded to ../Lab_4/doodles_images


In [4]:
doodles_path = '../Lab_4/doodles_images'

# Create an ImageDataGenerator for normalization
datagen = ImageDataGenerator(rescale=1./255)

# Load and preprocess the images
batch_size = 64
image_size = (64, 64)  

In [5]:
train_generator = datagen.flow_from_directory(
    doodles_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='input',  # Assuming all images belong to a single class
    shuffle=True
)

Found 0 images belonging to 0 classes.


In [ ]:
# Generator Model
def build_generator(latent_dim):
    model = models.Sequential()
    model.add(layers.Dense(8 * 8 * 256, input_dim=latent_dim))
    model.add(layers.Reshape((8, 8, 256)))
    model.add(layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv2D(3, (3, 3), activation='tanh', padding='same'))
    
    return model

In [ ]:
# Discriminator Model
def build_discriminator(image_size):
    model = models.Sequential()
    model.add(layers.Conv2D(64, (3, 3), strides=(2, 2), padding='same', input_shape=(image_size[0], image_size[1], 3)))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.4))
    model.add(layers.Conv2D(128, (3, 3), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.4))
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation='sigmoid'))
    
    return model

In [ ]:
# GAN Model
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = models.Sequential()
    model.add(generator)
    model.add(discriminator)
    model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5))
    
    return model

In [ ]:
# Build and compile the discriminator
discriminator = build_discriminator(image_size)
discriminator.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5), metrics=['accuracy'])

In [ ]:
# Build the generator
latent_dim = 100
generator = build_generator(latent_dim)

In [ ]:
# Build the GAN model
gan = build_gan(generator, discriminator)

In [ ]:
# Functiom to save generated images
def save_generated_images(epoch, generator, output_folder='generated_images', examples=10, dim=(1, 10), figsize=(10, 1)):
    noise = np.random.normal(0, 1, size=[examples, 100])
    generated_images = generator.predict(noise)
    generated_images = generated_images.reshape(examples, 64, 64, 3)

    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    plt.figure(figsize=figsize)
    for i in range(generated_images.shape[0]):
        plt.subplot(dim[0], dim[1], i + 1)
        plt.imshow((generated_images[i] + 1) / 2, interpolation='nearest')
        plt.axis('off')

        # Save each generated image as a JPEG file
        image_path = os.path.join(output_folder, f'generated_image_epoch_{epoch}_sample_{i}.jpg')
        plt.imsave(image_path, (generated_images[i] + 1) / 2)

    plt.tight_layout()
    plt.show()

In [ ]:
# Training the GAN model (you may need to adjust the number of epochs)
epochs = 5000
for epoch in range(epochs):
    for _ in range(train_generator.n // batch_size):
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        generated_images = generator.predict(noise)
        real_images, _ = train_generator.next()
        real_labels = np.ones((batch_size, 1))
        fake_labels = np.zeros((batch_size, 1))

        # Train the discriminator on real and fake images
        d_loss_real = discriminator.train_on_batch(real_images, real_labels)
        d_loss_fake = discriminator.train_on_batch(generated_images, fake_labels)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train the generator
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        valid_labels = np.ones((batch_size, 1))
        g_loss = gan.train_on_batch(noise, valid_labels)

    # Print the progress and save generated images
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, [D loss: {d_loss[0]}, acc.: {100 * d_loss[1]}], [G loss: {g_loss}]")

        # Save generated images
        if epoch % 500 == 0:
            save_generated_images(epoch, generator)